In [0]:
!pip install torchtext --upgrade

     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 1.0MB 10.3MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [1]:
%cd /content/drive/"My Drive"/in5550-exam

/content/drive/My Drive/in5550-exam


In [0]:
import argparse
import random
import torch
import torchtext
from torchtext import data
import NSR

In [0]:
import pickle
import os

In [0]:
args = argparse.Namespace(setupfile="Setup.json")
setup = NSR.Process(args)

In [0]:
RSEED = setup["randomseed"]
random.seed(RSEED)
torch.manual_seed(RSEED)
torch.backends.cudnn.deterministic = True

ID = data.RawField(preprocessing=lambda x: int(x))
SRC = data.RawField()
NEGS = data.RawField(preprocessing=lambda x: int(x))
FORM = data.Field(batch_first=True, include_lengths=True)
LEMMA = data.RawField()
XPOS = data.RawField()
LABS = data.Field(batch_first=True, unk_token=None)
CUE = data.Field(batch_first=True)
SCOPE = data.RawField()

fields = {
    "id": ("id", ID),
    "source": ("source", SRC),
    "negations": ("negations", NEGS),
    "form": ("form", FORM),
    "lemma": ("lemma", LEMMA),
    "xpos": ("xpos", XPOS),
    "negation": ("label", LABS),
    "cue": ("cue", CUE),
    "scope": ("scope", SCOPE),
}

Xtrain, Xdev = NSR.StarSEM2012.splits(
    "DataFiles",
    fields=fields,
    test=None
)


if setup["params"]["vectors"]:
  vecs = torchtext.vocab.Vectors(setup["params"]["vectors"])
else:
  vecs = None

FORM.build_vocab(Xtrain,
                  max_size=setup["params"]["vocab_size"],
                  min_freq=setup["params"]["min_freq"],
                  vectors=vecs)

CUE.build_vocab(Xtrain, vectors=vecs)

LABS.build_vocab(Xtrain)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vecs = FORM.vocab.vectors

model = setup.load("model", NSR.Models,
                    input_dim=len(FORM.vocab),
                    output_dim=len(LABS.vocab)
                    ).to(device)

optimizer = setup.load("optimizer", torch.optim, model.parameters())

criterion = setup.load("criterion", torch.nn, weight=None).to(device)

runner = setup.load("runner", NSR.Runners, model,
                    criterion, optimizer, LABS.vocab.itos)

batch_size = setup["params"]["batchsize"]
epochs = setup["params"]["epochs"]

trn_iter = data.BucketIterator(
    Xtrain,
    device=device,
    shuffle=True,
    batch_size=batch_size
)

val_iter = data.BucketIterator(
    Xdev,
    device=device,
    shuffle=True,
    batch_size=1
)


In [0]:
print(model)

BiLSTMC(
  (w_embedding): Embedding(5779, 50)
  (c_embedding): Embedding(5779, 50)
  (lstm): LSTM(50, 250, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=500, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


# 2. Training

In [0]:
runner.run(epochs, trn_iter, val_iter)

100%|██████████| 816/816 [00:02<00:00, 341.68it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
  2%|▏         | 6/315 [00:00<00:05, 56.83it/s]

{'accuracy': 0.8939181614349776, 'precision': 0.1787836322869955, 'recall': 0.2, 'macro_f1': 0.18879763226045135, 'confusion_matrix': [[0, 0, 0, 0, 0], [0, 12758, 0, 0, 0], [0, 1335, 0, 0, 0], [0, 146, 0, 0, 0], [0, 33, 0, 0, 0]]}


  2%|▏         | 5/315 [00:00<00:06, 49.27it/s]

{'accuracy': 0.9023963004484304, 'precision': 0.4230412213452345, 'recall': 0.35355292186137194, 'macro_f1': 0.3591004872190463, 'confusion_matrix': [[0, 0, 0, 0, 0], [0, 12758, 0, 0, 0], [0, 1325, 10, 0, 0], [0, 22, 13, 111, 0], [0, 12, 11, 10, 0]]}


  2%|▏         | 7/315 [00:00<00:05, 59.99it/s]

...

{'accuracy': 0.9676289237668162, 'precision': 0.7836229233348575, 'recall': 0.7314484020200989, 'macro_f1': 0.7533874294179033, 'confusion_matrix': [[0, 0, 0, 0, 0], [0, 12709, 49, 0, 0], [0, 412, 923, 0, 0], [0, 0, 0, 146, 0], [0, 1, 0, 0, 32]]}


  2%|▏         | 6/315 [00:00<00:05, 57.85it/s]

{'accuracy': 0.968329596412556, 'precision': 0.7738253969461558, 'recall': 0.7362999367551524, 'macro_f1': 0.7523385338489522, 'confusion_matrix': [[0, 0, 0, 0, 0], [0, 12684, 74, 0, 0], [0, 376, 958, 0, 1], [0, 0, 0, 146, 0], [0, 1, 0, 0, 32]]}


100%|██████████| 816/816 [00:02<00:00, 330.02it/s]

{'accuracy': 0.9626541479820628, 'precision': 0.780877961030376, 'recall': 0.7214823793790945, 'macro_f1': 0.745440644169225, 'confusion_matrix': [[0, 0, 0, 0, 0], [0, 12704, 54, 0, 0], [0, 478, 857, 0, 0], [0, 0, 0, 146, 0], [0, 1, 0, 0, 32]]}
Stopping after no improvement for 5 epochs


# 3. Dev/Validation Set Results

In [0]:
dev_res = runner.evaluate(val_iter)

100%|██████████| 816/816 [00:02<00:00, 330.63it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [0]:
print("*"*21)
print("{} {}".format(setup["identifier"], setup["ID"]))
print("Results:")
for i in ["accuracy", "precision", "recall", "macro_f1"]:
  print("\t{}: {:2f}".format(i, dev_res[i]))
print("Confusion Matrix: \n")
for i in dev_res["confusion_matrix"]:
  print(i)

*********************
BiLSTM-C Baseline_incr_hidd
Results:
	accuracy: 0.962654
	precision: 0.780878
	recall: 0.721482
	macro_f1: 0.745441
Confusion Matrix: 

[0, 0, 0, 0, 0]
[0, 12704, 54, 0, 0]
[0, 478, 857, 0, 0]
[0, 0, 0, 146, 0]
[0, 1, 0, 0, 32]


# 4. Save

In [0]:
with open(os.path.join(setup.destdir, "form"), "wb") as o:
  pickle.dump(FORM.vocab, o)

with open(os.path.join(setup.destdir, "cue"), "wb") as o:
  pickle.dump(CUE.vocab, o)

with open(os.path.join(setup.destdir, "labels"), "wb") as o:
  pickle.dump(LABS.vocab, o)

In [0]:
runner.save(dirpath=setup.destdir, checkpoint=True)

# Evaluator

In [0]:
import glob
import argparse
import random
import torch
import torchtext
from torchtext import data
import NSR
import pickle
import os
import json
from collections import OrderedDict, defaultdict
from pathlib import Path


class LabelMapper():
  
  def __init__(self, vocab_stoi, vocab_itos):
    self.label2ind = vocab_stoi
    self.labelset = vocab_itos
    
  def index2label(self, x):
    return self.labelset[x]
  
  def label2index(self, x):
    return self.label2ind[x]
        
  def __call__(self, x):
    if type(x) == int:
      self.index2label(x)
    elif type(x) == str:
      self.label2index(x)
    else:
      raise ValueError("This is not a valid key")


class Evaluator():
  """Use a saved model to evaluate on held out dataset.

  Loads a saved model and vocabularies to be used for
  predicting new data.
  """

  def __init__(self, saved_model_dir=None):
    self.work_dir = saved_model_dir
    
    setup_file = os.path.join(self.work_dir, "setup.json")
    with open(setup_file, "rt") as f:
        self.setup = json.load(f, object_hook=OrderedDict)
    
    with open(os.path.join(self.work_dir, "form"), "rb") as f:
      self.form_voc = pickle.load(f)
    
    with open(os.path.join(self.work_dir, "cue"), "rb") as f:
      self.cue_voc = pickle.load(f)
    
    with open(os.path.join(self.work_dir, "labels"), "rb") as f:
      self.lab_voc = pickle.load(f)
    
    modelfile = glob.glob(os.path.join(self.work_dir, "model_epoch*.pt"))[0]
    state_dict = torch.load(modelfile)["model_state_dict"]
    self.device = state_dict["fc.bias"].device
    
    model = self._load("model", NSR.Models, input_dim=len(self.form_voc),
                       output_dim=len(self.lab_voc)).to(self.device)
    
    model.load_state_dict(state_dict)
    self.model = model
    self.model.eval()

    self.FORM = data.Field(batch_first=True, include_lengths=True)
    self.LABS = data.Field(batch_first=True, unk_token=None)
    self.CUE = data.Field(batch_first=True)

    self.FORM.vocab = self.form_voc
    self.CUE.vocab = self.cue_voc
    self.LABS.vocab = self.lab_voc

    # Since the vocabs are not pickled as defaultdicts:
    self.FORM.vocab.stoi = defaultdict(torchtext.vocab._default_unk_index,
                                       self.FORM.vocab.stoi)
    self.CUE.vocab.stoi = defaultdict(torchtext.vocab._default_unk_index,
                                       self.CUE.vocab.stoi)

    self.mapp = LabelMapper(self.LABS.vocab.stoi,
                            self.LABS.vocab.itos)
    
    self.correction_log = []


  def map_node(self, id, f, l, x):
    return {"id": str(id), 
            "form": f, 
            "properties": {"lemma": l, "xpos": x}}
  
  def map_back_gold(self, sentence):
    out = sentence.copy()
  
    form = out.pop("form")
    lemma = out.pop("lemma")
    xpos = out.pop("xpos")
    cue = out.pop("cue")
    scope = out.pop("scope")
    negation = out.pop("negation")

    nodes = []

    if out["negations"] == 0:
      for i in range(len(form)):
        nodes.append(self.map_node(i, form[i], lemma[i], xpos[i]))
  
    else:
      for i in range(len(form)):
        node = self.map_node(i, form[i], lemma[i], xpos[i])
        if negation[i] == "T":
          node["negation"] = [{"id": 0}]
        elif negation[i] == "F":
          node["negation"] = [{"id": 0, "scope": scope[i]}]
        elif negation[i] == "C":
          node["negation"] = [{"id": 0, "cue": cue[i]}]
        elif negation[i] == "A":
          node["negation"] = [{"id": 0, "cue": cue[i], "scope": scope[i]}]
        else:
          node["negation"] = [{"id": 0}]

        nodes.append(node)

    out["nodes"] = nodes
    return out
  
  def map_back_pred(self, sentence):
    out = sentence.copy()
  
    form = out.pop("form")
    lemma = out.pop("lemma")
    xpos = out.pop("xpos")
    cue = out.pop("cue")
    scope = out.pop("scope")
    negation = out.pop("negation")

    nodes = []

    # To handle logic in the 'convert.py' file
    # need to set negations to 0

    if set(negation).issubset(set("T")):
      out["negations"] = 0
    else:
      out["negations"] = 1

    if out["negations"] == 0:
      for i in range(len(form)):
        nodes.append(self.map_node(i, form[i], lemma[i], xpos[i]))
  
    else:
      for i in range(len(form)):
        node = self.map_node(i, form[i], lemma[i], xpos[i])
        if negation[i] == "T":
          node["negation"] = [{"id": 0}]
        elif negation[i] == "F":
          node["negation"] = [{"id": 0, "scope": scope[i]}]
        elif negation[i] == "C":
          node["negation"] = [{"id": 0, "cue": cue[i]}]
        elif negation[i] == "A":
          node["negation"] = [{"id": 0, "cue": cue[i], "scope": scope[i]}]
        else:
          node["negation"] = [{"id": 0}]

        nodes.append(node)

    out["nodes"] = nodes
    return out  

  
  def _correct_preds(self, pred_labs, id, src):
    """Apply a simple constraint on the predictions."""

    n_cues = pred_labs.count("A") + pred_labs.count("C")

    # No predicted cues, but has scope
    if n_cues == 0 and not set(pred_labs).issubset(set("T")):
      # Correct everything to true
      out_labs = ["T"]*len(pred_labs)
      self.correction_log.append({"id": id, "source": src})
      return out_labs
    
    return pred_labs

  
  def pred(self, batch, apply_dk=False):
    with torch.no_grad():
      form_b = self.FORM.process([batch["form"]], device=self.device)
      cue_b = self.CUE.process([batch["cue"]], device=self.device)
      lab_b = self.LABS.process([batch["negation"]], device=self.device)

      y_tilde = self.model(form_b, cue_b)
      y_hat = y_tilde.argmax(dim=-1).cpu().flatten().tolist()
      y_hat = [self.mapp.index2label(l) for l in y_hat]
      if apply_dk:
        y_hat = self._correct_preds(y_hat, batch["id"], batch["source"])

      y_b = batch["negation"]
      return y_b, y_hat
  
  def output_preds(self, eval_file, results_file=None, apply_dk=False):
    if not results_file:
      results_file = "eval_pred.epe"

    out_file = os.path.join(self.work_dir, results_file)

    if os.path.isfile(out_file):
      raise Exception("There is already a file there")
    
    with open(eval_file) as f, open(out_file, "wt") as r:
      for line in f:
        sent = json.loads(line)
        _, y_pred = self.pred(sent, apply_dk)
        sent["negation"] = y_pred
        sent = self.map_back_pred(sent)
        json.dump(sent, r)
        r.write("\n")
    
    n_corr = len(self.correction_log)
    if apply_dk and n_corr > 0:
      with open(os.path.join(self.work_dir, "correction_log.txt"), "w") as log:
        for corr in self.correction_log:
          json.dump(corr, log)
          log.write("\n")

      print("{} corrections applied, see correction_log.txt".format(n_corr))
    
    print("File saved to {}".format(out_file))

  def output_preds_tidy(self, eval_file, results_file=None, apply_dk=False):
    if not results_file:
      results_file = "eval_pred_tidy.epe"

    out_file = os.path.join(self.work_dir, results_file)

    if os.path.isfile(out_file):
      raise Exception("There is already a file there")
    
    with open(eval_file) as f, open(out_file, "wt") as r:
      for line in f:
        sent = json.loads(line)
        _, y_pred = self.pred(sent, apply_dk)
        sent["negation"] = y_pred
        json.dump(sent, r)
        r.write("\n")
    
    print("File saved to {}".format(out_file))


  def _load(self, argname, module, *args, **kwargs):
    item = self.setup[argname]["type"]
    item_args = dict(self.setup[argname]["args"])
    if sum([kwarg in item_args for kwarg in kwargs]) != 0:
      raise ValueError("Args set in config file cannot be overwritten")
    item_args.update(kwargs)
    
    return getattr(module, item)(*args, **item_args)





Run evaluation

In [0]:
ev = Evaluator("Saves/BiLSTM-C/Baseline_incr_hidd")

In [20]:
ev.output_preds(eval_file="DataFiles/cde.epe", 
                results_file="eval_pred_final.epe",
                apply_dk=False)

File saved to Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred_final.epe


In [21]:
ev.output_preds(eval_file="DataFiles/cde.epe", 
                results_file="eval_pred_corr_final.epe",
                apply_dk=True)

1 corrections applied, see correction_log.txt
File saved to Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred_corr_final.epe


In [22]:
ev.output_preds_tidy(eval_file="DataFiles/cde.epe", 
                results_file="eval_pred_corr_tidy.epe",
                apply_dk=True)

File saved to Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred_corr_tidy.epe


In [23]:
ev.output_preds_tidy(eval_file="DataFiles/cde.epe", 
                results_file="eval_pred_tidy.epe",
                apply_dk=False)

File saved to Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred_tidy.epe


# Using `score.py` to evaluate:

In [0]:
import sys
sys.path.append("negation")

In [0]:
from negation import score, convert

In [18]:
%cd ..

/content/drive/My Drive/in5550-exam


Read the gold labels and predictions

In [0]:
s_gold = convert.read_negations("EvalFiles/eval_gold.epe")

In [0]:
s_pred = convert.read_negations("Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred_corr_final.epe")

In [0]:
file = "Baseline_incr_hidd_corr_score_final.txt"

In [27]:
%cd negation

/content/drive/My Drive/in5550-exam/negation


In [0]:
score.starsem_score(s_gold, s_pred, file)

In [0]:
os.chdir("")

***********************************************

# Testing

In [0]:
runner.labelvals

['<pad>', 'T', 'F', 'C', 'A']

In [0]:
res

{'accuracy': 0.09353979820627803,
 'confusion_matrix': [[0, 0, 0, 0, 0],
  [0, 0, 12758, 0, 0],
  [0, 0, 1335, 0, 0],
  [0, 0, 146, 0, 0],
  [0, 0, 33, 0, 0]],
 'macro_f1': 0.03421541615941565,
 'precision': 0.018707959641255607,
 'recall': 0.2}

### Dev

In [0]:
torch.load("Saves/BiLSTM-C/Baseline/checkpoint_stats")

{'eval': {'accuracy': 0.9513032511210763,
  'confusion_matrix': [[0, 0, 0, 0, 0],
   [0, 12542, 216, 0, 0],
   [0, 476, 859, 0, 0],
   [0, 0, 1, 144, 1],
   [0, 1, 0, 0, 32]],
  'macro_f1': 0.7296966771068604,
  'precision': 0.7462770697012415,
  'recall': 0.7165026958131212},
 'train': {'loss': 0.2042346529958584}}

Save Baseline

Save Baseline_incr_hidd

In [0]:
ev = Evaluator("Saves/BiLSTM-C/Baseline_incr_hidd")

In [0]:
ev.output_preds(eval_file="DataFiles/cde.epe", 
                results_file="eval_pred.epe",
                apply_dk=False)

File saved to Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred.epe


In [0]:
ev.output_preds(eval_file="DataFiles/cde.epe", 
                results_file="eval_pred_corr.epe",
                apply_dk=True)

3 corrections applied, see correction_log.txt
File saved to Saves/BiLSTM-C/Baseline_incr_hidd/eval_pred_corr.epe


In [0]:
import sys

In [0]:
sys.path.append("negation")

In [0]:
from negation import score

In [0]:
ev.FORM.vocab.stoi

In [0]:
inv_id = []
inv_src = []
for i in s:
  gold, pred = ev.pred(i)
  if "A" in gold or "C" in gold:
    print("Gold: {}".format(gold))
    print("Pred: {}".format(pred))
    print("*"*21)
  if pred.count("A") + pred.count("C") > 1 or gold.count("A") + gold.count("C") > 1:
    if gold.count("A") + gold.count("C") > 1:
      inv_id.append(i["id"])
      inv_src.append(i["source"])
    print("\t\n\n" + "#"*21)
    print(i["form"])
    print("id: ", i["id"], "\t", "source:", i["source"] )
    print("\t\tGold: {}".format(gold))
    print("\t\tPred: {}".format(pred))


Gold: ['F', 'F', 'C', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
Pred: ['F', 'F', 'C', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
*********************
Gold: ['C', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T']
Pred: ['C', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T']
*********************
Gold: ['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'C', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T']
Pred: ['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'C', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'T']
*********************
Gold: ['T', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'A', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
Pred: ['T', 'T', 'T', 'T', 'T', 'T'

In [0]:
# 1. Read the eval file
with open("DataFiles/cde.epe") as d_in, \
     open("EvalFiles/eval_gold.epe", "wt") as out:
  for line in d_in:
    sent = json.loads(line)
    sent = ev.map_back_gold(sent)
    json.dump(sent, out)
    out.write("\n")


In [0]:
# 2. Eval and print to file



In [0]:
with open("EvalFiles/eval_gold.epe") as f:
  for line in f:
    print(line)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
invs = list(zip(inv_id, inv_src))

In [0]:
with open("DataFiles/cdd.epe") as f:
  s = [json.loads(line) for line in f]

In [0]:
with open("negation/cdd.epe") as f:
  s2 = [json.loads(line) for line in f]

In [0]:
for i in s2:
  if (i["id"], i["source"]) in invs:
    print("\t", i["negations"])
    for n in i["nodes"]:
      print("\t\t", n["form"], "\tNegation: ", end="\n")
      for l in n.get("negation"):
        print("\t\t\t", l, end="\n")
    print("*"*31)

	 2
		 The 	Negation: 
			 {'id': 25, 'scope': 'The'}
			 {'id': 26}
		 dinner 	Negation: 
			 {'id': 25, 'scope': 'dinner'}
			 {'id': 26}
		 itself 	Negation: 
			 {'id': 25, 'scope': 'itself'}
			 {'id': 26}
		 was 	Negation: 
			 {'id': 25, 'scope': 'was'}
			 {'id': 26}
		 neither 	Negation: 
			 {'id': 25, 'cue': 'neither'}
			 {'id': 26}
		 well 	Negation: 
			 {'id': 25, 'scope': 'well'}
			 {'id': 26}
		 served 	Negation: 
			 {'id': 25, 'scope': 'served', 'event': 'served'}
			 {'id': 26}
		 nor 	Negation: 
			 {'id': 25, 'cue': 'nor'}
			 {'id': 26}
		 well 	Negation: 
			 {'id': 25, 'scope': 'well'}
			 {'id': 26}
		 cooked 	Negation: 
			 {'id': 25, 'scope': 'cooked', 'event': 'cooked'}
			 {'id': 26}
		 , 	Negation: 
			 {'id': 25}
			 {'id': 26}
		 and 	Negation: 
			 {'id': 25}
			 {'id': 26}
		 the 	Negation: 
			 {'id': 25}
			 {'id': 26, 'scope': 'the'}
		 gloomy 	Negation: 
			 {'id': 25}
			 {'id': 26, 'scope': 'gloomy'}
		 presence 	Negation: 
			 {'id': 25}
			 {

In [0]:
def map_node(id, form, lemma, xpos):
  return {"id": str(id), 
          "form": form, 
          "properties": {"lemma": lemma, "xpos": xpos}}

def map_back(sentence):
  out = sentence.copy()
  
  form = out.pop("form")
  lemma = out.pop("lemma")
  xpos = out.pop("xpos")
  cue = out.pop("cue")
  scope = out.pop("scope")
  negation = out.pop("negation")

  nodes = []

  if sentence["negations"] == 0:
    for i in range(len(form)):
      nodes.append(map_node(i, form[i], lemma[i], xpos[i]))
  
  else:
    for i in range(len(form)):
      node = map_node(i, form[i], lemma[i], xpos[i])
      if negation[i] == "T":
        node["negation"] = [{"id": 0}]
      elif negation[i] == "F":
        node["negation"] = [{"id": 0, "scope": scope[i]}]
      elif negation[i] == "C":
        node["negation"] = [{"id": 0, "cue": cue[i]}]
      elif negation[i] == "A":
        node["negation"] = [{"id": 0, "cue": cue[i], "scope": scope[i]}]
      else:
        node["negation"] = [{"id": 0}]

      nodes.append(node)

  out["nodes"] = nodes
  return out

def pred_map_back(sentence):
  out = sentence.copy()
  
  form = out.pop("form")
  lemma = out.pop("lemma")
  xpos = out.pop("xpos")
  cue = out.pop("cue")
  scope = out.pop("scope")
  negation = out.pop("negation")

  nodes = []

  if sentence["negations"] == 0:
    for i in range(len(form)):
      nodes.append(map_node(i, form[i], lemma[i], xpos[i]))
  
  else:
    for i in range(len(form)):
      node = map_node(i, form[i], lemma[i], xpos[i])
      if negation[i] == "T":
        node["negation"] = [{"id": 0}]
      elif negation[i] == "F":
        node["negation"] = [{"id": 0, "scope": scope[i]}]
      elif negation[i] == "C":
        node["negation"] = [{"id": 0, "cue": cue[i]}]
      elif negation[i] == "A":
        node["negation"] = [{"id": 0, "cue": cue[i], "scope": scope[i]}]
      else:
        node["negation"] = [{"id": 0}]

      nodes.append(node)

  out["nodes"] = nodes
  return out


In [0]:
s4 = map_back(s[4])

In [0]:
s4

{'id': '4',
 'negations': 0,
 'nodes': [{'form': 'Suddenly',
   'id': '0',
   'properties': {'lemma': 'Suddenly', 'xpos': 'RB'}},
  {'form': 'he', 'id': '1', 'properties': {'lemma': 'he', 'xpos': 'PRP'}},
  {'form': 'turned',
   'id': '2',
   'properties': {'lemma': 'turn', 'xpos': 'VBD'}},
  {'form': 'upon', 'id': '3', 'properties': {'lemma': 'upon', 'xpos': 'IN'}},
  {'form': 'me', 'id': '4', 'properties': {'lemma': 'me', 'xpos': 'PRP'}},
  {'form': 'with', 'id': '5', 'properties': {'lemma': 'with', 'xpos': 'IN'}},
  {'form': 'a', 'id': '6', 'properties': {'lemma': 'a', 'xpos': 'DT'}},
  {'form': 'mischievous',
   'id': '7',
   'properties': {'lemma': 'mischievous', 'xpos': 'JJ'}},
  {'form': 'twinkle',
   'id': '8',
   'properties': {'lemma': 'twinkle', 'xpos': 'NN'}},
  {'form': 'in', 'id': '9', 'properties': {'lemma': 'in', 'xpos': 'IN'}},
  {'form': 'his', 'id': '10', 'properties': {'lemma': 'his', 'xpos': 'PRP$'}},
  {'form': 'eyes', 'id': '11', 'properties': {'lemma': 'eye', 'x

In [0]:
def test_map(sent):
  has_c_cue = "C" in sent["negation"]
  has_a_cue = "A" in sent["negation"]
  if has_c_cue or has_a_cue:
    

{'cue': ['', '', '', '', '', '', '', '', ''],
 'form': ['1.',
  'The',
  'Singular',
  'Experience',
  'of',
  'Mr.',
  'John',
  'Scott',
  'Eccles'],
 'id': '0',
 'lemma': ['1.',
  'The',
  'Singular',
  'Experience',
  'of',
  'Mr.',
  'John',
  'Scott',
  'Eccles'],
 'negation': ['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T'],
 'negations': 0,
 'scope': ['', '', '', '', '', '', '', '', ''],
 'source': 'wisteria01',
 'xpos': ['CD', 'NNP', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP']}

In [0]:
if 2:
  print(1)

1


In [0]:
import tqdm

In [0]:
y_hat = torch.tensor([], dtype=torch.long)
y = torch.tensor([], dtype=torch.long)

tp = 0
n = 0

model.eval()
with torch.no_grad():
    for batch in tqdm.tqdm(val_iter):
        y_tilde_b = model(batch.form)
        y_hat_b = y_tilde_b.argmax(dim=-1)
        y_b = batch.label

        tp += (y_hat_b == y_b).nonzero().size(0)
        n += y_b.size(1)

        y_hat = torch.cat((y_hat, y_hat_b.view(-1).cpu()))
        y = torch.cat((y, y_b.view(-1).cpu()))

results_eval = {
    "accuracy": tp / n,
    **runner.get_metrics(y, y_hat)
}

100%|██████████| 816/816 [00:01<00:00, 485.68it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [0]:
tss = set(["T", "A"]*20)

In [0]:
set(["T"]*20).issubset(set("T"))

True

### Multiclass confmat

In [0]:
from sklearn.metrics import multilabel_confusion_matrix

In [0]:
multilabel_confusion_matrix(y, y_hat, labels=runner.labels)

In [0]:
xxx = next(iter(val_iter))

In [0]:
TEXT.process(train.examples[0].text)[0].shape

In [0]:
xproc = FORM.process([Xtrain.examples[1].form])
cproc = CUE.process([Xtrain.examples[1].cue])

In [0]:
mx = model.embedding(xproc[0].to(device))
mc = model.embedding(cproc[0].to(device))

In [0]:
(mx + mc).shape

torch.Size([1, 33, 300])

In [0]:
xproc[1] == cproc[1]

tensor([True])

In [0]:
from collections import Counter

In [0]:
import json

In [0]:
c = Counter()
with open("DataFiles/cdt.epe") as f:
  for line in f:
    for l in json.loads(line)["negation"]:
      c[l] += 1
    


In [0]:
cdev = Counter()
with open("DataFiles/cdd.epe") as f:
  for line in f:
    for l in json.loads(line)["negation"]:
      cdev[l] += 1

In [0]:
c

Counter({'A': 159, 'C': 843, 'F': 6802, 'T': 61351})

In [0]:
cdev

Counter({'A': 33, 'C': 146, 'F': 1335, 'T': 12758})

In [0]:
LABS.vocab.itos

['<pad>', 'T', 'F', 'C', 'A']

In [0]:
torch.nn.CrossEntropyLoss()

In [0]:
(1/159) + (1/843) + (1/6802) + (1/61351)

0.007638863032973212

In [0]:
y1 = 159
y2 = 843
y3 = 6802
y4 = 61351

In [0]:
(y1*y2*y3*y4)/(y2*y3*y4 + y1*y3*y4 + y1*y2*y4 + y1*y2*y3)

130.9095340083324

In [0]:
w1 = 130.91/159
w2 = 130.91/843
w3 = 130.91/6802
w4 = 130.91/61351

In [0]:
w1+w2+w3+w4

1.0000035596465233

In [0]:
ID = data.RawField(preprocessing=lambda x: int(x))
SRC = data.RawField()
NEGS = data.RawField(preprocessing=lambda x: int(x))
FORM = data.Field(batch_first=True, include_lengths=True)
LEMMA = data.RawField()
XPOS = data.RawField()
LABS = data.Field(batch_first=True, unk_token=None)
CUE = data.Field(batch_first=True)
SCOPE = data.RawField()

fields = {
    "id": ("id", ID),
    "source": ("source", SRC),
    "negations": ("negations", NEGS),
    "form": ("form", FORM),
    "lemma": ("lemma", LEMMA),
    "xpos": ("xpos", XPOS),
    "negation": ("label", LABS),
    "cue": ("cue", CUE),
    "scope": ("scope", SCOPE),
}

Xtrain, Xdev = NSR.StarSEM2012.splits(
    "DataFiles",
    fields=fields,
    test=None
)

FORM.build_vocab(Xtrain,
                  max_size=None,
                  min_freq=1,
                  vectors=None)

In [0]:
FORM.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             ',': 2,
             '.': 3,
             'the': 4,
             'I': 5,
             'of': 6,
             'and': 7,
             '``': 8,
             "''": 9,
             'to': 10,
             'a': 11,
             'that': 12,
             'in': 13,
             'it': 14,
             'was': 15,
             'you': 16,
             'he': 17,
             'is': 18,
             'his': 19,
             'have': 20,
             '?': 21,
             'had': 22,
             'not': 23,
             'my': 24,
             'with': 25,
             'which': 26,
             'for': 27,
             'as': 28,
             'we': 29,
             'me': 30,
             'at': 31,
             'be': 32,
             'upon': 33,
             'this': 34,
             'him': 35,
             'but': 36,
             'from': 37,
             'said': 38,
             'no': 39,
   

In [0]:
FORM.vocab._default_unk_index

AttributeError: ignored

In [0]:
torchtext.vocab._default_unk_index

<function torchtext.vocab._default_unk_index>

In [0]:
with open